# Hopfield Network

Hopfield Network 是一种经典的**循环神经网络（Recurrent Neural Network, RNN）**，最早由 John Hopfield 在1982年提出，用于模拟**人脑的联想记忆（associative memory）功能**。它是最早与物理系统（尤其是统计力学）相结合的神经网络之一，启发了现代神经网络和能量模型的发展（如 Boltzmann Machine 和现代 Attention 的 Hopfield reinterpretation）。

---

## 一、Hopfield Network 的核心概念

### 特点：

* 是一种**完全连接的对称网络**
* 是**无监督学习模型**
* 可用于**模式存储和回忆**（pattern storage & retrieval）
* 动态行为基于**能量函数**：网络总朝着能量最小的状态演化

---

## 二、网络结构与表示

设：

* 网络共有 $N$ 个神经元，每个神经元的状态为 $s_i \in \{-1, +1\}$
* 神经元之间有连接权重 $w_{ij} \in \mathbb{R}$，满足：

  $$
  w_{ii} = 0 \quad \text{(无自连接)}, \quad w_{ij} = w_{ji} \quad \text{(对称连接)}
  $$

---

## 三、动态更新规则（异步更新）

![](./image/1.png)

每次选择一个神经元 $i$，按以下方式更新它的状态：

$$
s_i^{(t+1)} = \text{sign}\left( \sum_{j=1}^{N} w_{ij} s_j^{(t)} \right)
$$

其中 $\text{sign}(x)$ 表示符号函数：

$$
\text{sign}(x) =
\begin{cases}
+1 & x > 0 \\
-1 & x < 0 \\
s_i^{(t)} & x = 0 \quad (\text{状态不变})
\end{cases}
$$

---

## 四、能量函数（Energy Function）

Hopfield 网络的关键在于它的**能量函数**，定义为：

$$
E(s) = -\frac{1}{2} \sum_{i,j} w_{ij} s_i s_j
$$

### 性质：

* 每次更新都会降低网络能量或保持不变
* 网络在有限步后**稳定于局部最小能量态**
* 这些能量最小态对应**记忆存储的模式**

---

## 五、模式学习：Hebbian 规则（存储记忆）

若要存储 $P$ 个模式 $\{ \xi^\mu \}_{\mu=1}^P$，每个 $\xi^\mu \in \{-1, +1\}^N$，权重矩阵用 Hebb 规则定义为：

$$
w_{ij} = \frac{1}{N} \sum_{\mu=1}^{P} \xi_i^\mu \xi_j^\mu, \quad \text{其中 } w_{ii} = 0
$$

这个规则使得这些模式变成了能量函数的**稳定点**。

---

## 六、工作机制总结（联想记忆）

1. **存储阶段**：利用 Hebb 规则构造权重矩阵
2. **回忆阶段**：给定一个受扰模式作为初始状态
3. **动态演化**：每次选择一个神经元更新状态
4. **收敛结果**：最终网络稳定于某个吸引态（可能是原始模式）

---

## 七、网络容量

Hopfield 网络的容量并不是无限的。经典结果：

$$
P_{\text{max}} \approx 0.138 \cdot N
$$

也就是说，当存储的模式数大于这个上限时，网络会出现“干扰”（cross-talk），不能稳定回忆。




In [ ]:
import torch

class HopfieldNetwork:
    def __init__(self, num_neurons):
        self.num_neurons = num_neurons
        self.weights = torch.zeros((num_neurons, num_neurons))

    def store_patterns(self, patterns):
        # patterns: [num_patterns, num_neurons], values in {-1, +1}
        for pattern in patterns:
            self.weights += torch.outer(pattern, pattern)
        self.weights.fill_diagonal_(0)
        self.weights /= self.num_neurons

    def retrieve(self, state, steps=10):
        # state: [num_neurons], initial state (may be noisy)
        state = state.clone()
        for _ in range(steps):
            i = torch.randint(0, self.num_neurons, (1,))
            net_input = torch.dot(self.weights[i], state)
            state[i] = torch.sign(net_input) if net_input != 0 else state[i]
        return state
